In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.5 MB/s eta 0:00:00


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [6]:
skincare = pd.read_csv(f"{DATA_PATH}아모레크롤링_스킨케어.csv")

In [7]:
skincare = skincare.drop(columns=['상품분류'])

In [8]:
makeup = pd.read_csv(f"{DATA_PATH}메이크업 별점.csv")

In [9]:
food = pd.read_csv(f"{DATA_PATH}뷰티푸드 별점.csv")

In [10]:
perfume = pd.read_csv(f"{DATA_PATH}향수 별점.csv")

In [11]:
thing = pd.read_csv(f"{DATA_PATH}소품도구 별점.csv")

In [ ]:
thing

,브랜드명,상품명,사용자 이름,별점,리뷰작성날짜,나이,성별,피부타입,피부트러블,리뷰
0,헤라,블랙 쿠션 제로-비티 핏 퍼프 2EA,ogy4***,2,2021.10.29,20대,여성,NaN,NaN,-디자인\n\n*5점 만점 기준*\n\n-디자인\n: 퍼프 위의 가죽 느낌으로 고급...
1,헤라,블랙 쿠션 제로-비티 핏 퍼프 2EA,bjhd**,5,2022.02.17,30대,남성,NaN,NaN,좋아요! 최고에요!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!...
2,헤라,블랙 쿠션 제로-비티 핏 퍼프 2EA,kitt*****,5,2022.07.19,20대,여성,NaN,NaN,퍼프는 빨아서 사용하는 것도 찝찝하고 블랙쿠션에는 블랙쿠션 퍼프가 딱이라고 생각해서...
3,헤라,블랙 쿠션 제로-비티 핏 퍼프 2EA,sjji****,5,2021.08.14,50대 이상,여성,NaN,NaN,퍼프가 발림성이 정말로 쫀쫀해요 쫀쫀 !\n두 개의 퍼프가 비닐과 pvc백에 이중 ...
4,헤라,블랙 쿠션 제로-비티 핏 퍼프 2EA,wlsg********,5,2022.07.20,30대,여성,NaN,NaN,"밀착력에 짱 ! 뭉침없이 발라지는 퍼프, 저는 위생상 2주일에 한번씩 직접 퍼프를 ..."
...,...,...,...,...,...,...,...,...,...,...
18603,피카소,[피카소] 103A 파우더,suy*,5,2019.08.07,20대,여성,NaN,NaN,명불허전이에요~\n파우더나 블러셔 용도로 매우 잘쓰고 있어요!
18604,피카소,[피카소] 103A 파우더,yyou****,5,2019.06.25,20대,여성,NaN,NaN,부드럽고 파우더 바르기 좋아요!\n다른 브러쉬로 바르면 파우더가 조금 떠서 피카소 ...
18605,피카소,[피카소] 103A 파우더,amil****,5,2019.05.18,40대,여성,NaN,NaN,피카소브러쉬 너무좋아요 계속사게되요 모질도 너무 부드럽고 ~ \n이번에 행사해서 할...
18606,피카소,[피카소] 103A 파우더,vbgu*****,4,2019.04.30,30대,여성,NaN,NaN,가격은 좀 비싸지만 생각보다 괜찮은것 같아요\n배송은 하루만에 받았구요 보호케이스가...


In [12]:
frames = [skincare, makeup, food, perfume, thing]
df = pd.concat(frames, ignore_index=True)
df

,브랜드명,상품명,사용자 이름,별점,리뷰작성날짜,나이,성별,피부타입,피부트러블,리뷰
0,이니스프리,화산송이 모공 바하 클렌징 폼 150g,0103*******,5,2022.09.27,40대,여성,지성,모공,남편이 극지성이에요. 원래 수퍼화산송이 미셀라만 꾸준히 사용해 오다가 지성 전용이라...
1,이니스프리,화산송이 모공 바하 클렌징 폼 150g,lsm2***,5,2022.08.15,40대,여성,건성,건조함,여름이라 집에 와서 메이크업 클렌징 해줄때 꼭 이중세안 해주는데 이 폼 클렌징으로 ...
2,이니스프리,화산송이 모공 바하 클렌징 폼 150g,gpdl***,5,2022.12.24,20대,여성,복합성,트러블,이니스프리 폼클렌징은 피부에 자극적이지 않아 좋아요 선크림만 사용해도 꼭 폼까지 써...
3,이니스프리,화산송이 모공 바하 클렌징 폼 150g,zoll***,5,2022.09.30,30대,여성,건성,건조함,저희 엄마가 이 제품이 클렌징이 제일 잘된다고 좋아하서 1+1 이벤트하길래 4개 쟁...
4,이니스프리,화산송이 모공 바하 클렌징 폼 150g,snow****,5,2022.09.30,50대 이상,여성,지성,모공,여름이라서 모공이 넓어지는것 같아서 구매해서 사용해보니 촉촉하고 피부땡기지 않아서 ...
...,...,...,...,...,...,...,...,...,...,...
220177,피카소,[피카소] 103A 파우더,suy*,5,2019.08.07,20대,여성,NaN,NaN,명불허전이에요~\n파우더나 블러셔 용도로 매우 잘쓰고 있어요!
220178,피카소,[피카소] 103A 파우더,yyou****,5,2019.06.25,20대,여성,NaN,NaN,부드럽고 파우더 바르기 좋아요!\n다른 브러쉬로 바르면 파우더가 조금 떠서 피카소 ...
220179,피카소,[피카소] 103A 파우더,amil****,5,2019.05.18,40대,여성,NaN,NaN,피카소브러쉬 너무좋아요 계속사게되요 모질도 너무 부드럽고 ~ \n이번에 행사해서 할...
220180,피카소,[피카소] 103A 파우더,vbgu*****,4,2019.04.30,30대,여성,NaN,NaN,가격은 좀 비싸지만 생각보다 괜찮은것 같아요\n배송은 하루만에 받았구요 보호케이스가...


In [13]:
def chatbot(df):
    model_name = "noahkim/KoT5_news_summarization"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    while True:
        title = input("상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): ")
        if title.lower() in ['q', 'quit']:
            print("챗봇을 종료합니다.")
            break
        # 여러 키워드에 대한 처리
        keywords = title.split()
        condition = lambda x: all(keyword.lower() in x.lower() for keyword in keywords)
        # 중복 제거를 위해 unique() 사용
        matching_titles = [t for t in df['상품명'].unique().tolist() if condition(t)]
        if not matching_titles:
            print("해당 상품이 없습니다.")
            continue
        print("일치하는 상품명 목록:")
        for idx, matching_title in enumerate(matching_titles, start=1):
            print(f"{idx}. {matching_title}")
        # 사용자가 상품을 선택하도록 함
        selected_idx = int(input("상품을 선택하세요 (번호 입력): "))
        selected_title = matching_titles[selected_idx - 1]
        # 선택한 상품의 리뷰 가져오기
        reviews = df[df['상품명'] == selected_title]['리뷰'].to_list()
        reviews_text = ' '.join(reviews)
        # 리뷰 요약
        inputs = tokenizer(reviews_text, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs["input_ids"], num_beams=4, min_length=40, max_length=200, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        print(f"{selected_title}의 전체 리뷰 요약: {summary}")

In [18]:
chatbot(df)

상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): 화산송이
일치하는 상품명 목록:
1. 화산송이 모공 바하 클렌징 폼 150g
2. 수퍼 화산송이 모공 마스크 100ml
3. 화산송이 모공 바하 클렌징폼  250g
상품을 선택하세요 (번호 입력): 1
화산송이 모공 바하 클렌징 폼 150g의 전체 리뷰 요약: 여름이라 집에 와서 메이크업 클렌징 해줄때 꼭 이중세안 해주는데 이 폼 클렌징으로 세안 해주면 좋은점이 뾰루지가 안나고 뽀드득하게 씻겨줘서 좋고 피지과도할때 딥 클렌징폼이라 그런지 피부에 자극이 없는건 아니지만 격일로 쓰기에 좋은거같아요.
상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): q
챗봇을 종료합니다.


In [14]:
import re

In [15]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 49.5 MB/s eta 0:00:00


In [16]:
from konlpy.tag import Okt
from collections import Counter

In [17]:
def keyword_bot(df):
    while True:
        title = input("상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): ")
        if title.lower() in ['q', 'quit']:
            print("챗봇을 종료합니다.")
            break
        matching_titles = df[df['상품명'].str.contains(title, case=False)]['상품명'].unique().tolist()
        if not matching_titles:
            print("해당 상품이 없습니다.")
            continue
        print("일치하는 상품명 목록:")
        for idx, matching_title in enumerate(matching_titles, start=1):
            print(f"{idx}. {matching_title}")
        selected_idx = int(input("상품을 선택하세요 (번호 입력): "))
        selected_title = matching_titles[selected_idx - 1]
        keyword(selected_title)  # 상품명을 함수에 전달하여 실행

def keyword(title):
    texts = df[df['상품명'] == title]['리뷰'].to_list()
    okt = Okt()
    nouns_list = []
    for text in texts:
        nouns = okt.nouns(text)
        nouns_list.extend(nouns)

    count = Counter(nouns_list)

    # 명사 빈도 카운트
    noun_list = count.most_common(100)
    for v in noun_list:
        if any(keyword in v[0] for keyword in ['지성','수부지', '건성', '복합성','여드름','잡티','피부톤','탄력','피지','모공','블랙헤드','민감성','봄웜','웜톤','봄웜톤','봄라이트','봄라','봄브라이트','여쿨','여름쿨톤','여라','여름라이트','여름뮤트','여뮽','뮤트','뮤트톤','여름브라이트','가을웜톤','가을뮤트','갈딥','가을딥','갈웜','겨쿨','겨울브라이트','겨울딥','쿨톤']):
            print("이 제품은 "+v[0]+" 타입에게 추천하는 제품입니다.")

In [19]:
keyword_bot(df)

상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): 화산송이
일치하는 상품명 목록:
1. 화산송이 모공 바하 클렌징 폼 150g
2. 수퍼 화산송이 모공 마스크 100ml
3. 화산송이 모공 바하 클렌징폼  250g
상품을 선택하세요 (번호 입력): 1
이 제품은 모공 타입에게 추천하는 제품입니다.
이 제품은 피지 타입에게 추천하는 제품입니다.
이 제품은 여드름 타입에게 추천하는 제품입니다.
이 제품은 건성 타입에게 추천하는 제품입니다.
상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): q
챗봇을 종료합니다.


In [ ]:
keyword_bot(df)

상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): 블러셔
일치하는 상품명 목록:
1. 하트 팝 블러셔 4g
2. 러블리쿠키 블러셔
3. [피카소] 생기창조 108 블러셔
4. [피카소꼴레지오니] 160 팬 블러셔
5. [피카소꼴레지오니] 108 블러셔
6. [피카소] PONY14 블러셔
상품을 선택하세요 (번호 입력): 1
상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): 2
일치하는 상품명 목록:
1. 글루타 비타민C 토닝 앰플 23g
2. 오가니언스 BR 소프트 필링 투 폼 클렌저 120ml
3. 오가니언스 워터리 에센스 230ml
4. 에이시카 365 수분 진정 결 토너 250ml
5. 어린쑥 진정맑은 클렌징 폼 120g
6. 멜라솔브 프로그램 딥 클렌징 폼 200g
7. 그린티 아미노 수분 클렌징 워터 320ml
8. 레티놀 엑스퍼트 0.3% 20ml
9. 슈퍼바이탈 기초 2종 세트
10. 0.2 테라피 에어 마스크  20ml
11. 오가니언스 베리어 리페어 2종 기획 세트
12. 백화고 클렌징 마사지 크림 250ml
13. 모이스트 클렌징 오일 200ml
14. 순행클렌징오일 200ml
15. 로즈워터 토너 250ml
16. 자음생아이크림 단품세트 20ml
17. 에이시카 365 흔적 진정 세럼 25ml
18. UV 쉴드 선 스틱 SPF50+/PA++++ 20g
19. 페이셜 마일드 필링 250ml
20. 맨 바이오 안티에이징 2종 세트
21. 하이드로 리플렉팅 2종 세트
22. 순행클렌징폼 200ml
23. 송담 탄력 에멀젼 125ml
24. 아토베리어365 하이드로 에센스 200ml
25. 어린쑥 수분진정 2종 세트
26. 블랙티 유스 인핸싱 앰플 마스크 28ml
27. 프레쉬 모이스춰 클렌징폼 120g
28. 바하 버블 필링 클렌저 200ml
29. 자음유액 125ml
30. 마일드 클렌징폼 200g
31. 립 앤 아이 리무버 250ml
32. 어린쑥 진정맑은 클렌징 오일 200ml
33. 백화고 아이크림 25ml